<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/deepseekr1_api_demo_june2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://platform.deepseek.com/usage

In [2]:
!pip3 install openai -q

In [4]:
from google.colab import userdata

import os
deepseek_api_key = userdata.get('DEEPSEEK_API_KEY')

* The deepseek-chat model points to DeepSeek-V3-0324. You can invoke it by specifying model='deepseek-chat'.

* The deepseek-reasoner model points to DeepSeek-R1-0528. You can invoke it by specifying model='deepseek-reasoner'

https://huggingface.co/deepseek-ai/DeepSeek-R1-0528?utm_campaign=The%20Batch&utm_medium=email&_hsenc=p2ANqtz--M3NQ_S2i2a4-YigM_g7yAl65J52ZDkfTOAs4rKcQS5b4sBcaw8xHQxWdvtC9-m5_DNtdk92DS5nC5r5inVdTsTJvNZQ&_hsmi=364910056&utm_content=364905475&utm_source=hs_email

In [7]:
# Please install OpenAI SDK first: `pip3 install openai`

from openai import OpenAI

client = OpenAI(api_key=deepseek_api_key, base_url="https://api.deepseek.com")

response = client.chat.completions.create(
    #model="deepseek-chat",
    model="deepseek-reasoner",
    messages=[
        {"role": "system", "content": "You are a helpful assistant"},
        {"role": "user", "content": "Hello"},
    ],
    stream=False
)

print(response.choices[0].message.content)

Hello! 😊 How can I help you today?


In [16]:
def deepseek_chat(prompt,model):
  response = client.chat.completions.create(
    #model="deepseek-chat",
    #model="deepseek-reasoner",
    model=model,
    messages=[
        {"role": "system", "content": "You are a helpful assistant"},
        {"role": "user", "content": prompt},
    ],
    stream=False
  )

  print()
  print("-" * 80)
  print('Model: %s'%model)
  print("-" * 80)
  print('\n')

  print("-" * 80)
  print('Question: %s'%prompt)
  print("-" * 80)
  print('\n')

  print('Answer: ')
  print(response.choices[0].message.content)
  #return message.content[0].text
  return response.choices[0].message.content

In [18]:
prompt = "What is the capital of France?"
response = deepseek_chat(prompt,'deepseek-chat')
print(response)


--------------------------------------------------------------------------------
Model: deepseek-chat
--------------------------------------------------------------------------------


--------------------------------------------------------------------------------
Question: What is the capital of France?
--------------------------------------------------------------------------------


Answer: 
The capital of France is **Paris**. 

Paris is known for its iconic landmarks such as the Eiffel Tower, the Louvre Museum, and the Notre-Dame Cathedral. It is also a major global hub for art, fashion, gastronomy, and culture. 

Let me know if you'd like more details!
The capital of France is **Paris**. 

Paris is known for its iconic landmarks such as the Eiffel Tower, the Louvre Museum, and the Notre-Dame Cathedral. It is also a major global hub for art, fashion, gastronomy, and culture. 

Let me know if you'd like more details!


In [19]:
prompt= 'How do you plan out your trip? \
Bob is travelling to SAT from YVR \
1. He has a connection in DFW \
2. His connection is 6 hours long \
3. He has a budget of 100.00 including meals \
4. What can he do? Please suggest a time. \
5. Know- he is a hiker, museum, foodie, has a carry-on bag'

response = deepseek_chat(prompt,"deepseek-reasoner")
print(response)


--------------------------------------------------------------------------------
Model: deepseek-reasoner
--------------------------------------------------------------------------------


--------------------------------------------------------------------------------
Question: How do you plan out your trip? Bob is travelling to SAT from YVR 1. He has a connection in DFW 2. His connection is 6 hours long 3. He has a budget of 100.00 including meals 4. What can he do? Please suggest a time. 5. Know- he is a hiker, museum, foodie, has a carry-on bag
--------------------------------------------------------------------------------


Answer: 
Based on Bob's interests (hiking, museums, food), budget ($100), and constraints (6-hour layover at DFW with carry-on only), here's a tailored plan:

### **Recommended Itinerary:**
**Total Time Needed:** 4–5 hours (allowing 1–1.5 hours for security re-entry)  
**Budget Allocation:** $45–$65 (covers transit, activities, and a meal)  
**Focus:** Quick 

## TTP AGENT

Cell 1: Install Necessary Libraries

In [ ]:
!pip install crewai -q
!pip install 'crewai[tools]' -q
!pip install openai -q

Cell 2: Set Up DeepSeek API Key

In [21]:
from google.colab import userdata
import os

# Retrieve the DeepSeek API Key from Colab's userdata secrets
deepseek_api_key = userdata.get('DEEPSEEK_API_KEY')

if not deepseek_api_key:
    raise ValueError("DEEPSEEK_API_KEY not found in Colab secrets. Please add it.")

DEEPSEEK_API_KEY = deepseek_api_key # Assign to the variable used in subsequent cells

Cell 3: Define a Custom Tool for DeepSeek Interaction (using BaseTool)

In [40]:
from crewai.tools import BaseTool
from openai import OpenAI
from pydantic import Field

class DeepSeekChatTool(BaseTool):
    name: str = "DeepSeekChatTool"
    description: str = "A powerful AI model for generating responses based on prompts."

    client: OpenAI = Field(default=None, exclude=True)

    # Add model_name as a field to be accessible in _run
    model_name: str = Field(default='deepseek-reasoner')

    def __init__(self, api_key: str, model_name: str = 'deepseek-reasoner', tool_name: str = "DeepSeek Reasoner", tool_description: str = "A powerful AI model capable of complex reasoning and generating detailed plans, useful for creating travel itineraries and solving logistical problems.", **data):
        # Ensure tool_name and tool_description are passed to the superclass
        super().__init__(name=tool_name, description=tool_description, **data)

        # Initialize client and model_name
        self.client = OpenAI(api_key=api_key, base_url="https://api.deepseek.com")
        self.model_name = model_name # Store model_name

    def _run(self, prompt: str) -> str:
        """
        Interacts with the DeepSeek model to get a response based on the prompt.
        """

        #print(f"Debug: Tool received prompt of type: {type(prompt)}")
        #print(f"Debug: Tool received prompt value: {prompt}")

        if not isinstance(prompt, str):
             # If the prompt is not a string, try to find the relevant text
             # This is a heuristic and might need adjustment based on what's actually passed
             if isinstance(prompt, dict) and 'description' in prompt:
                 actual_prompt = prompt['description']
                 #print(f"Debug: Extracted prompt from dictionary: {actual_prompt}")
             else:
                 #print("Debug: Could not extract string prompt from input.")
                 # Re-raise the error or handle it appropriately
                 raise ValueError(f"Expected prompt to be a string, but received {type(prompt)}")
        else:
            actual_prompt = prompt


        response = self.client.chat.completions.create(
            model=self.model_name, # Use the stored model_name
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": actual_prompt}, # Use the potentially extracted prompt
            ],
            stream=False
        )
        return response.choices[0].message.content

# Instantiate the tool with model_name being explicitly passed
deepseek_reasoner_tool = DeepSeekChatTool(
    api_key=DEEPSEEK_API_KEY,
    model_name='deepseek-reasoner',
    tool_name="DeepSeek Reasoner",
    tool_description="A powerful AI model capable of complex reasoning and generating detailed plans, useful for creating travel itineraries and solving logistical problems."
)

deepseek_chat_tool = DeepSeekChatTool(
    api_key=DEEPSEEK_API_KEY,
    model_name='deepseek-chat',
    tool_name="DeepSeek Chat",
    tool_description="A general-purpose AI model for conversational tasks and information retrieval."
)

Cell 4: Define Your Agents

In [44]:
from crewai import Agent

# Agent 1: Itinerary Planner
itinerary_planner = Agent(
    role='Expert Travel Itinerary Creator',
    goal='Create comprehensive and personalized tourism travel itineraries based on user interests, budget, and time constraints.',
    backstory="You are a seasoned travel agent with a knack for crafting perfect trips. You use advanced AI to generate detailed plans.",
    tools=[deepseek_reasoner_tool], # This agent needs the reasoning capability
    verbose=False
)

# Agent 2: Local Activities Recommender
local_activities_recommender = Agent(
    role='Local Expert and Activity Specialist',
    goal='Suggest relevant local activities and attractions that align with traveler preferences and layover constraints.',
    backstory="You know all the hidden gems and popular spots in any city. You ensure the traveler makes the most of their time.",
    tools=[deepseek_reasoner_tool], # Can also use the reasoner for more nuanced suggestions
    verbose=False
)

# Agent 3: Budget and Logistics Manager
budget_logistics_manager = Agent(
    role='Financial and Logistical Advisor',
    goal='Manage the travel budget and provide practical logistical advice for transportation and timing.',
    backstory="You are meticulous with numbers and timings, ensuring the trip stays within budget and runs smoothly.",
    tools=[deepseek_reasoner_tool], # Reasoning is key for budget allocation and timing
    verbose=False
)

Cell 5: Define Your Tasks

In [45]:
from crewai import Task

# Task for the Itinerary Planner
plan_itinerary_task = Task(
    description=(
        "Generate a detailed tourism travel itinerary for a traveler with the following profile:\n"
        "- Departing from YVR, connecting in DFW for 6 hours, final destination SAT.\n"
        "- Budget: $100 including meals.\n"
        "- Interests: hiker, museum lover, foodie.\n"
        "- Has a carry-on bag only.\n"
        "The itinerary should include recommended activities, budget breakdown, and a clear timeline, "
        "as demonstrated in the example plan from the DeepSeek API demo for Bob's trip. "
        "Ensure the output format is markdown."
    ),
    agent=itinerary_planner,
    output_file='dfw_layover_itinerary.md', # Save the output
    expected_output="A markdown formatted, comprehensive tourism travel itinerary including timeline, budget, and activities for a DFW layover based on the provided user profile, similar to the example in the DeepSeek API demo."
)

Cell 6: Form the Crew and Run

In [46]:
from crewai import Crew, Process

crew = Crew(
    agents=[itinerary_planner, local_activities_recommender, budget_logistics_manager],
    tasks=[plan_itinerary_task],
    verbose=False, # Increased verbosity for detailed logs
    process=Process.sequential # Agents execute tasks one after another
)

print("Starting the DFW Layover Tourism Planning Crew...")
result = crew.kickoff()
print("\n\n########################")
print("## DFW Layover Itinerary ##")
print("########################\n")
print(result)

Starting the DFW Layover Tourism Planning Crew...


########################
## DFW Layover Itinerary ##
########################

### DFW Layover Itinerary: Vancouver (YVR) → Dallas (DFW) → San Antonio (SAT)  
**Layover Duration:** 6 hours  
**Interests:** Hiking, Museums, Food  
**Budget:** $100 (meals + activities)  
**Key Constraints:** Immigration clearance (YVR→DFW is international), security re-entry, and luggage recheck. *Leaving the airport is risky due to time limits; this itinerary focuses on secure, in-airport experiences.*  

---

### **Timeline & Activities**  
*(Assuming arrival at DFW Terminal D at 10:00 AM; next flight at 4:00 PM)*  

#### **10:00 AM – 11:30 AM: Arrival & Immigration**  
- Deplane and clear U.S. immigration/customs (allow 60–90 mins).  
- Recheck luggage for SAT flight.  
- *Pro Tip:* Use Mobile Passport Control app to expedite queues.  

#### **11:30 AM – 1:00 PM: Food & Exploration**  
- **Lunch at Whitetail Bistro (Terminal D, Gate D40)**  
  - Texa